#Recommendation system on hindi movies


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#### Loading the Dataset
The data was scraped from IMDb's website using Python libraries such as requests, BeautifulSoup, and Pandas. <br>About
The Dataset features 2200 Bollywood movies, sorted by popularity from 2023 to 1951.

In [4]:
df = pd.read_csv("nupur1_db.csv")

Vizualizing Dataframe




In [5]:
df.head()

,Unnamed: 0,movie_id,movie_name,year,genre,overview,director,cast,cleaned_overview
0,0,tt15354916,Jawan,2023,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ...",highoctane action thriller outline emotional j...
1,1,tt15748830,Jaane Jaan,2023,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ...",single mother daughter commit crime neighbor h...
2,2,tt11663228,Jailer,2023,"Action, Comedy, Crime",A retired jailer goes on a manhunt to find his...,Nelson Dilipkumar,"Rajinikanth, Mohanlal, Shivarajkumar, Jackie S...",retired jailer go manhunt find son killer road...
3,3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,2023,"Comedy, Drama, Family",Flamboyant Punjabi Rocky and intellectual Beng...,Karan Johar,"Ranveer Singh, Alia Bhatt, Dharmendra, Shabana...",flamboyant punjabi rocky intellectual bengali ...
4,4,tt15732324,OMG 2,2023,"Comedy, Drama",An unhappy civilian asks the court to mandate ...,Amit Rai,"Pankaj Tripathi, Akshay Kumar, Yami Gautam, Pa...",unhappy civilian asks court mandate comprehens...


In [6]:
df.describe()

,Unnamed: 0
count,2199.000000
mean,1099.369259
std,635.344495
min,0.000000
25%,549.500000
50%,1099.000000
75%,1649.500000
max,2199.000000


In [7]:
print(df.columns.values)

['Unnamed: 0' 'movie_id' 'movie_name' 'year' 'genre' 'overview' 'director'
 'cast' 'cleaned_overview']


In [8]:
df.rename(columns={'genre': 'genres', 'movie_name': 'title' , 'cleaned_overview':'keywords'}, inplace=True)

df

,Unnamed: 0,movie_id,title,year,genres,overview,director,cast,keywords
0,0,tt15354916,Jawan,2023,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ...",highoctane action thriller outline emotional j...
1,1,tt15748830,Jaane Jaan,2023,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ...",single mother daughter commit crime neighbor h...
2,2,tt11663228,Jailer,2023,"Action, Comedy, Crime",A retired jailer goes on a manhunt to find his...,Nelson Dilipkumar,"Rajinikanth, Mohanlal, Shivarajkumar, Jackie S...",retired jailer go manhunt find son killer road...
3,3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,2023,"Comedy, Drama, Family",Flamboyant Punjabi Rocky and intellectual Beng...,Karan Johar,"Ranveer Singh, Alia Bhatt, Dharmendra, Shabana...",flamboyant punjabi rocky intellectual bengali ...
4,4,tt15732324,OMG 2,2023,"Comedy, Drama",An unhappy civilian asks the court to mandate ...,Amit Rai,"Pankaj Tripathi, Akshay Kumar, Yami Gautam, Pa...",unhappy civilian asks court mandate comprehens...
...,...,...,...,...,...,...,...,...,...
2194,2195,tt11112474,Heeriye,NaN,Thriller,Add a Plot,Subhash Ghai,"Shatrughan Sinha, Reena Roy, Ajit Khan, Premna...",add plot
2195,2196,tt0332766,Sur: The Melody of Life,2002,"Drama, Musical, Romance",A renowned music teacher mentors a promising y...,Tanuja Chandra,"Lucky Ali, Simone Singh, Achint Kaur, Ehsan Khan",renowned music teacher mentor promising young ...
2196,2197,tt8622232,Time to Dance,2021,"Musical, Romance",When a ballroom dancer's shot at a crucial tou...,Stanley D'Costa,"Sooraj Pancholi, Isabelle Kaif, Waluscha D'Sou...",ballroom dancer shot crucial tournament jeopar...
2197,2198,tt0187351,Nigahen: Nagina Part II,1989,"Drama, Family, Fantasy",After the tragic deaths of his son Ajit and da...,Harmesh Malhotra,"Sunny Deol, Sridevi, Anupam Kher, Gulshan Grover",tragic death son ajit daughterinlaw rajni raja...


Removing extra information that we do not need

In [9]:
features = ['genres', 'keywords', 'title', 'cast', 'director']

Some columns have NaN data points that may create a problem.<br>Replacing NaN values with empty string ('').

In [10]:
df['cast'].isnull().values.any()

False

Creating a function for combining the values of these columns into a single string

In [11]:
def combine_features(row):
    return row['title']+' '+row['genres']+' '+row['director']+' '+row['keywords']+' '+row['cast']

In [12]:
for feature in features:
    df[feature] = df[feature].fillna('')

applying combine_feature method over each row of Dataframe and storing the combined string in "combined_features" column

In [13]:
df['combined_features'] = df.apply(combine_features, axis = 1)

In [14]:
print(df.loc[0, 'combined_features'])

Jawan Action, Thriller Atlee highoctane action thriller outline emotional journey man set rectify wrong society Shah Rukh Khan, Nayanthara, Vijay Sethupathi, Deepika Padukone



Feeding the combined strings to a CountVectorizer() object for getting the count matrix.

In [15]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['combined_features'])

Obtaining the cosine similarity matrix from the count matrix.

In [16]:
cosine_sim = cosine_similarity(count_matrix)

defining two helper functions to get movie title from movie index and vice-versa.

In [17]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

* Get the title of the movie that the user currently likes.<br>



* Find the index of that movie. <br>
* Access the row corresponding to this movie in the similarity matrix. Thus, we will get the similarity scores of all other movies from the current movie. <br>
* Enumerate through all the similarity scores of that movie to make a tuple of movie index and similarity score. <br><br>
This will convert a row of similarity scores like this- [1 0.5 0.2 0.9] to this- [(0, 1) (1, 0.5) (2, 0.2) (3, 0.9)] . Here, each item is in this form- (movie index, similarity score)

In [19]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
    # Instead of using an "index" column, use the DataFrame's index directly.
    # This assumes your titles are in a column named 'title' and you want the corresponding index.
    return df[df.title == title].index.values[0]

movie_user_likes = "Kalank"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it

We will sort the list similar_movies according to similarity scores in descending order. Since the most similar movie to a given movie will be itself, we will discard the first element after sorting the movies.

In [20]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

Then, we will run a loop to print first 5 entries from sorted_similar_movies list.

In [21]:
i=0
print("Top 10 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>10:
        break

Top 10 similar movies to Kalank are:

Sadak 2
Housefull 5
Darlings
2 States
Dear Zindagi
Haseena Maan Jaayegi
Saawariya
Baiju Bawra
Panipat
Shaandaar
Lust Stories
